In [10]:
import requests
import pandas as pd
import datetime
from datetime import datetime
import numpy as np
import sqlite3

In [11]:
url_groupdetail = "https://api.purpleair.com/v1/groups/1662"
url_newmember = "https://api.purpleair.com/v1/groups/1662/members"
url_sensordata = "https://api.purpleair.com/v1/sensors/39355"
url_groupdata = "https://api.purpleair.com/v1/groups/1662/members"
url_deletemember = "https://api.purpleair.com/v1/groups/1662/members/158862"

key_get = "C111944A-B3A7-11ED-B6F4-42010A800007"
key_post = "C112A231-B3A7-11ED-B6F4-42010A800007"
headers = {'X-API-Key': key_get}

### Create SQL Database

In [12]:
conn = sqlite3.connect('air-pollution.db')
c = conn.cursor()

c.execute('''
          CREATE TABLE IF NOT EXISTS air_pollution
          ([sensor_index] INTEGER PRIMARY KEY, [pm_25] REAL, [pm_10] REAL, [temp_celsius] INTEGER,
          [date] TEXT)
          ''')

conn.commit()

In [13]:
# Fetch data 
conn = sqlite3.connect('air-pollution.db') 
c = conn.cursor()
                   
c.execute('''
          SELECT
          sensor_index, pm_25, pm_10, temp_celsius, date FROM air_pollution;
          ''')

df = pd.DataFrame(c.fetchall(), columns=['sensor_index', 'pm_25', 'pm_10', 'temp_celsius', 'date'])
df

,sensor_index,pm_25,pm_10,temp_celsius,date


### Sensor: 34641

In [14]:
url_34641 = "https://api.purpleair.com/v1/sensors/34641"

In [15]:
payload_pm25_34641 = {'fields': 'pm2.5'}
payload_pm10_34641 = {'fields': 'pm10.0'}
payload_temp_34641 = {'fields':  'temperature'}

In [16]:
sensor_index_34641 = 34641

pm25_34641 = requests.get(url_34641, headers = headers, params = payload_pm25_34641)
pm25_34641 = pm25_34641.json()
pm25_34641 = pm25_34641['sensor']['pm2.5']

pm10_34641 = requests.get(url_34641, headers = headers, params = payload_pm10_34641)
pm10_34641 = pm10_34641.json()
pm10_34641 = pm10_34641['sensor']['pm10.0']

temp_34641 = requests.get(url_34641, headers = headers, params = payload_temp_34641)
temp_34641 = temp_34641.json()
temp_34641 = temp_34641['sensor']['temperature']

now = datetime.now()
date_34641 = now.strftime("%d/%m/%Y %H:%M:%S")

In [17]:
c.execute('INSERT INTO air_pollution(sensor_index, pm_25, pm_10, temp_celsius, date) VALUES(?, ?, ?, ?, ?)',
               (sensor_index_34641, pm25_34641, pm10_34641, temp_34641, date_34641))

conn.commit()
conn.close()

In [18]:
# Fetch data 
conn = sqlite3.connect('air-pollution.db') 
c = conn.cursor()
                   
c.execute('''
          SELECT
          sensor_index, pm_25, pm_10, temp_celsius, date FROM air_pollution;
          ''')

df_1 = pd.DataFrame(c.fetchall(), columns=['sensor_index', 'pm_25', 'pm_10', 'temp_celsius', 'date'])
df_1

,sensor_index,pm_25,pm_10,temp_celsius,date
0,34641,17.0,19.0,95,02/03/2023 18:48:34


### List of sensors in San Pedro

In [443]:
sensors = requests.get(url_groupdetail, headers=headers)

In [444]:
sensors = sensors.json()

In [445]:
sensors = sensors['members']

In [446]:
sensors

[{'id': 158850, 'sensor_index': 34641, 'created': 1677195580},
 {'id': 158851, 'sensor_index': 37747, 'created': 1677196464},
 {'id': 158849, 'sensor_index': 38537, 'created': 1677195516},
 {'id': 158854, 'sensor_index': 143372, 'created': 1677196569},
 {'id': 160198, 'sensor_index': 39355, 'created': 1677264653},
 {'id': 158855, 'sensor_index': 39697, 'created': 1677196598},
 {'id': 158852, 'sensor_index': 34569, 'created': 1677196500},
 {'id': 158853, 'sensor_index': 39731, 'created': 1677196526},
 {'id': 158858, 'sensor_index': 147010, 'created': 1677196677},
 {'id': 158859, 'sensor_index': 140440, 'created': 1677196694},
 {'id': 158856, 'sensor_index': 173083, 'created': 1677196625},
 {'id': 158857, 'sensor_index': 45769, 'created': 1677196650},
 {'id': 158863, 'sensor_index': 121825, 'created': 1677196842},
 {'id': 158860, 'sensor_index': 34511, 'created': 1677196728},
 {'id': 158861, 'sensor_index': 49563, 'created': 1677196755},
 {'id': 158866, 'sensor_index': 37657, 'created': 

In [447]:
sensors = pd.DataFrame(sensors)

In [448]:
sensors = sensors[['sensor_index', 'id', 'created']]

#sensors.rename(columns = {'sensor_index':'index_label'}, inplace = True)

In [449]:
sensors.rename(columns = {'id': 'group_id'}, inplace = True)

In [450]:
sensors = sensors.drop('created', axis=1)

In [451]:
sensors

,sensor_index,group_id
0,34641,158850
1,37747,158851
2,38537,158849
3,143372,158854
4,39355,160198
5,39697,158855
6,34569,158852
7,39731,158853
8,147010,158858
9,140440,158859


### PM 2.5

In [406]:
payload_pm25 = {'fields': 'pm2.5'}

In [407]:
pm25 = requests.get(url_groupdata, headers = headers, params = payload_pm25)

In [408]:
pm25 = pm25.json()

In [409]:
pm25 = pm25['data']

In [410]:
pm25 = pd.DataFrame(pm25, columns = ('sensor_index', 'pm_25'))

In [411]:
pm25.head()

,sensor_index,pm_25
0,140432,2.5
1,140440,3.7
2,143372,8.2
3,147010,5.4
4,33117,3.6


### PM 10

In [412]:
payload_pm10 = {'fields': 'pm10.0'}

In [413]:
pm10 = requests.get(url_groupdata, headers = headers, params = payload_pm10)

In [414]:
pm10 = pm10.json()

In [415]:
pm10 = pm10['data']

In [416]:
pm10 = pd.DataFrame(pm10, columns = ('sensor_index', 'pm_10'))

In [417]:
pm10.head()

,sensor_index,pm_10
0,140432,3.1
1,140440,4.0
2,143372,10.4
3,147010,6.5
4,33117,3.9


### Temperature

In [418]:
payload_temp = {'fields': 'temperature'}

In [419]:
temp = requests.get(url_groupdata, headers = headers, params = payload_temp)

In [420]:
temp = temp.json()

In [421]:
temp = temp['data']

In [422]:
temp = pd.DataFrame(temp, columns = ('sensor_index', 'temperature_f'))

In [423]:
# Convert fahrenheit to celsius.
temp['temp_celsius'] = np.round((temp["temperature_f"] - 32) * .5556, decimals = 0)
temp['temp_celsius'] = temp['temp_celsius'].astype(int)

# Delete fahrenheit column.
temp = temp.drop('temperature_f', axis=1)

In [424]:
temp.head()

,sensor_index,temp_celsius
0,140432,34
1,140440,37
2,143372,38
3,147010,41
4,33117,39


### Date

In [425]:
now = datetime.now()

In [426]:
timestamp = now.strftime("%d/%m/%Y %H:%M:%S")

In [427]:
timestamp = pd.Series(timestamp)

In [428]:
timestamp = pd.DataFrame(timestamp)

In [429]:
timestamp = pd.DataFrame(np.repeat(timestamp.values, 32, axis=0))

In [430]:
timestamp.head()

,0
0,02/03/2023 12:53:09
1,02/03/2023 12:53:09
2,02/03/2023 12:53:09
3,02/03/2023 12:53:09
4,02/03/2023 12:53:09


### Join data into one database

In [431]:
air_pollution = pd.concat([sensors.set_index('sensor_index'),pm25.set_index('sensor_index')], axis=1, join='inner').reset_index()

In [432]:
air_pollution = pd.concat([air_pollution.set_index('sensor_index'), pm10.set_index('sensor_index')], axis=1, join='inner').reset_index()

In [433]:
air_pollution = pd.concat([air_pollution.set_index('sensor_index'), temp.set_index('sensor_index')], axis=1, join='inner').reset_index()

In [434]:
air_pollution['date'] = timestamp

In [435]:
air_pollution

,sensor_index,group_id,pm_25,pm_10,temp_celsius,date
0,34641,158850,6.8,7.3,38,02/03/2023 12:53:09
1,37747,158851,8.8,11.5,36,02/03/2023 12:53:09
2,38537,158849,12.2,13.7,38,02/03/2023 12:53:09
3,143372,158854,8.2,10.4,38,02/03/2023 12:53:09
4,39355,160198,7.3,9.6,39,02/03/2023 12:53:09
5,39697,158855,6.4,7.9,36,02/03/2023 12:53:09
6,34569,158852,5.7,6.3,42,02/03/2023 12:53:09
7,39731,158853,6.1,7.1,40,02/03/2023 12:53:09
8,147010,158858,5.4,6.5,41,02/03/2023 12:53:09
9,140440,158859,3.7,4.0,37,02/03/2023 12:53:09


In [436]:
air_pollution.dtypes

sensor_index      int64
group_id          int64
pm_25           float64
pm_10           float64
temp_celsius      int64
date             object
dtype: object